In [3]:
import os
import json
import shutil

# -----------------------------
# Función de copia eficiente
# -----------------------------
def copy_large_file(src, dst, buffer_size=1024*1024*50):  # 50 MB por bloque
    if not os.path.exists(src):
        print(f"[Aviso] Archivo no encontrado: {src}")
        return
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    with open(src, "rb") as fsrc, open(dst, "wb") as fdst:
        shutil.copyfileobj(fsrc, fdst, buffer_size)

# -----------------------------
# Rutas
# -----------------------------
BASE_DIR = "/media/guest/PORT-DISK/Practicas/MicroBleeds/VALDO/Task2"  # carpeta original
NNUNET_DIR = "/media/guest/PORT-DISK/Practicas/Microbleeds/nnUNet_raw_data/TaskVALDO"  # destino nnU-Net

# Crear carpetas destino
imagesTr_dir = os.path.join(NNUNET_DIR, "imagesTr")
labelsTr_dir = os.path.join(NNUNET_DIR, "labelsTr")
os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)

# -----------------------------
# Recorrer sujetos
# -----------------------------
subject_folders = sorted([f for f in os.listdir(BASE_DIR)
                          if f.startswith("sub") and os.path.isdir(os.path.join(BASE_DIR, f))])

for sub_folder in subject_folders:
    subject_path = os.path.join(BASE_DIR, sub_folder)
    sub_id = sub_folder.replace("sub", "")  # solo número

    # Archivos originales
    img_t2s = os.path.join(subject_path, f"{sub_folder}_space-T2S_desc-masked_T2S.nii.gz")
    img_t1  = os.path.join(subject_path, f"{sub_folder}_space-T2S_desc-masked_T1.nii.gz")
    img_t2  = os.path.join(subject_path, f"{sub_folder}_space-T2S_desc-masked_T2.nii.gz")
    lbl_cmb = os.path.join(subject_path, f"{sub_folder}_space-T2S_CMB.nii.gz")

    # Archivos destino nnU-Net (multi-canal)
    img_dst = os.path.join(imagesTr_dir, f"{sub_folder}_0000.nii.gz")  # T2S
    img_dst1 = os.path.join(imagesTr_dir, f"{sub_folder}_0001.nii.gz")  # T1
    img_dst2 = os.path.join(imagesTr_dir, f"{sub_folder}_0002.nii.gz")  # T2
    lbl_dst = os.path.join(labelsTr_dir, f"{sub_folder}.nii.gz")        # máscara

    # Copiar archivos usando la función optimizada
    copy_large_file(img_t2s, img_dst)
    copy_large_file(img_t1, img_dst1)
    copy_large_file(img_t2, img_dst2)
    copy_large_file(lbl_cmb, lbl_dst)

# -----------------------------
# Crear dataset.json para nnU-Net (3 canales)
# -----------------------------
dataset_json = {
    "name": "VALDO_Task2",
    "description": "Microbleeds detection from VALDO Task2",
    "tensorImageSize": "3D",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "modality": {
        "0": "T2S",
        "1": "T1",
        "2": "T2"
    },
    "labels": {
        "0": "background",
        "1": "CMB"
    },
    "numTraining": len(subject_folders),
    "numTest": 0,
    "training": [{"image": f"./imagesTr/{sub}_0000.nii.gz",
                  "label": f"./labelsTr/{sub}.nii.gz"} for sub in subject_folders],
    "test": []
}

with open(os.path.join(NNUNET_DIR, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=4)

print("✅ Conversión completada. Estructura lista para nnU-Net con 3 canales (T2S, T1, T2).")


✅ Conversión completada. Estructura lista para nnU-Net con 3 canales (T2S, T1, T2).
